In [1]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np
import os

TensorFlow 2.x selected.


In [2]:
if not os.path.exists('tok.valid.abstract.txt'):
  !wget -O 'tok.valid.abstract.txt' 'https://www.dropbox.com/s/0c6e9yf8yhf9a75/tok.valid.abstract.txt?dl=1'
if not os.path.exists('tok.valid.body.txt'):
  !wget -O 'tok.valid.title.txt' 'https://www.dropbox.com/s/aiy87847kusb7ju/tok.valid.title.txt?dl=1'

--2019-12-08 13:10:07--  https://www.dropbox.com/s/0c6e9yf8yhf9a75/tok.valid.abstract.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/0c6e9yf8yhf9a75/tok.valid.abstract.txt [following]
--2019-12-08 13:10:07--  https://www.dropbox.com/s/dl/0c6e9yf8yhf9a75/tok.valid.abstract.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf9bb924bc399881be2f9ce503f.dl.dropboxusercontent.com/cd/0/get/At339IDFUWVvlppSEKekGjo4SvJ5w3v0B2JtnNKvAKSnHVvafffLqNfg_GWGIfHX0PwlfLmnKvg68iejMrhXIKvpF0rqYJzmCk1r1nQ92yTrlQRxSs8hfzZTdtHFtbsjF1Y/file?dl=1# [following]
--2019-12-08 13:10:08--  https://ucf9bb924bc399881be2f9ce503f.dl.dropboxusercontent.com/cd/0/get/At339IDFUWVvlppSEKekGjo4SvJ5w3v0B2JtnNKvAKSnHVvafffLqNfg_GWGIfHX0PwlfLmnKvg68ie

In [0]:
with open('tok.valid.abstract.txt','rb') as f:
    body_data = f.read().decode("utf-8").split('\n')
    
with open('tok.valid.title.txt','rb') as f:
    target_data = f.read().decode("utf-8").split('\n')

In [54]:
!pip install py-rouge

import nltk
nltk.download('punkt')
import rouge

def prepare_results(metric, p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


def print_rouge(hypo, ref):

    for aggregator in ['Avg']:# 'Best', 'Individual'
        print('Evaluation with {}'.format(aggregator))
        apply_avg = aggregator == 'Avg'
        apply_best = aggregator == 'Best'

        evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l'],
                            max_n=2,
                            limit_length=True,
                            length_limit=100,
                            length_limit_type='words',
                            apply_avg=apply_avg,
                            apply_best=apply_best,
                            alpha=0.5, # Default F1_score
                            weight_factor=1.2,
                            stemming=True)
        
        all_hypothesis = hypo
        all_references = ref

        scores = evaluator.get_scores(all_hypothesis, all_references)

        for metric, results in sorted(scores.items(), key=lambda x: x[0]):
            if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
                for hypothesis_id, results_per_ref in enumerate(results):
                    nb_references = len(results_per_ref['p'])
                    for reference_id in range(nb_references):
                        print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                        print('\t' + prepare_results(metric, results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
                print()
            else:
                print(prepare_results(metric, results['p'], results['r'], results['f']))
        print()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
for n in range(3):
    print("Lead-{}".format(n + 1))
    print_rouge(['.'.join(body_data[i].split(".")[:n+1]) for i in range(len(target_data))], target_data)
    print()

print("Last-1")
print_rouge(['.'.join(body_data[i].split(".")[-2:]) for i in range(len(target_data))], target_data)

Lead-1
Evaluation with Avg
	rouge-1:	P: 26.31	R: 45.31	F1: 31.57
	rouge-2:	P: 11.26	R: 20.10	F1: 13.64
	rouge-l:	P: 26.40	R: 41.45	F1: 31.00


Lead-2
Evaluation with Avg
	rouge-1:	P: 18.33	R: 59.62	F1: 27.03
	rouge-2:	P:  7.81	R: 26.68	F1: 11.62
	rouge-l:	P: 18.93	R: 51.12	F1: 26.85


Lead-3
Evaluation with Avg
	rouge-1:	P: 14.65	R: 67.75	F1: 23.43
	rouge-2:	P:  6.42	R: 31.15	F1: 10.33
	rouge-l:	P: 15.62	R: 56.80	F1: 23.95


Last-1
Evaluation with Avg
	rouge-1:	P: 19.33	R: 33.73	F1: 22.93
	rouge-2:	P:  5.54	R: 10.42	F1:  6.82
	rouge-l:	P: 20.09	R: 31.89	F1: 23.38

